In [21]:
import json
import os
from datetime import datetime

In [22]:
conll_path = '/home/burak/Desktop/thesis/code/IM/conll_data'
twitter_path = '/home/burak/Desktop/thesis/code/IM/twitter_partitioned/partition_105615'
output_path = '/home/burak/Desktop/thesis/code/IM/data_to_feed_to_the_model'

def mix_data(conll_path, twitter_path, output_path, conll_token_count=None, partition_label=None, conll_in_dev=False):
    # array of 3 arrays: train, test, dev
    conll_parses = [{},{},{}]
    conll_relations = [[], [], []]
    twitter_parses = [{},{},{}]
    twitter_relations = [[],[],[]]
    total_parses = [{}, {}, {}]
    total_relations = [[], [], []]
    with open(conll_path + '/en.train/parses.json') as conll_train_parses_file:
        with open(conll_path + '/en.train/relations.json') as conll_train_relations_file:
            conll_train_wsj_files = json.loads(conll_train_parses_file.read())
            conll_train_relations = conll_train_relations_file.readlines()
            if conll_token_count != None:
                cur_token_count = 0
                for c_t_k in conll_train_wsj_files.keys():
                    file_token_count = 0
                    c_t_v = conll_train_wsj_files[c_t_k]
                    for sentence in c_t_v['sentences']:
                        token_count = len(sentence['words'])
                        file_token_count += token_count
                    cur_token_count += file_token_count
                    conll_parses[0][c_t_k] = c_t_v
                    total_parses[0][c_t_k] = c_t_v
                    if cur_token_count > conll_token_count:
                        break
                print(conll_parses[0].keys())
                for relation in conll_train_relations:
                    cur_relation = json.loads(relation)
                    if cur_relation['DocID'] in conll_parses[0].keys():
                        conll_relations[0].append(cur_relation)
                        total_relations[0].append(cur_relation)
                print('TOKEN COUNT: ', cur_token_count)
            else:
                total_parses[0].update(conll_train_wsj_files)
                [total_relations[0].append(json.loads(relation)) for relation in conll_train_relations]

    if conll_in_dev:
        with open(conll_path + '/en.dev/parses.json') as conll_train_parses_file:
            with open(conll_path + '/en.dev/relations.json') as conll_train_relations_file:
                conll_train_wsj_files = json.loads(conll_train_parses_file.read())
                conll_train_relations = conll_train_relations_file.readlines()
                total_parses[2].update(conll_train_wsj_files)
                total_relations[2] = [json.loads(relation) for relation in conll_train_relations]
    
    # twitter
    with open(twitter_path + '/en.train/parses.json') as twitter_train_parses_file:
        with open(twitter_path + '/en.train/relations.json') as twitter_train_relations_file:
            twitter_train_thread_files = json.loads(twitter_train_parses_file.read())
            twitter_train_relations = twitter_train_relations_file.readlines()
            total_parses[0].update(twitter_train_thread_files)
            [total_relations[0].append(json.loads(relation)) for relation in twitter_train_relations]

    
    with open(twitter_path + '/en.test/parses.json') as twitter_train_parses_file:
        with open(twitter_path + '/en.test/relations.json') as twitter_train_relations_file:
            twitter_train_thread_files = json.loads(twitter_train_parses_file.read())
            twitter_train_relations = twitter_train_relations_file.readlines()
            total_parses[1].update(twitter_train_thread_files)
            total_relations[1] = [json.loads(relation) for relation in twitter_train_relations]
    
    with open(twitter_path + '/en.dev/parses.json') as twitter_train_parses_file:
        with open(twitter_path + '/en.dev/relations.json') as twitter_train_relations_file:
            twitter_train_thread_files = json.loads(twitter_train_parses_file.read())
            twitter_train_relations = twitter_train_relations_file.readlines()
            total_parses[2].update(twitter_train_thread_files)
            total_relations[2] = [json.loads(relation) for relation in twitter_train_relations]
    
    now = datetime.now()
    current_time = now.strftime("%Y%m%d%H%M%S")

    partition_path = output_path + '/partition_' + current_time
    if partition_label != None:
        partition_path += '_' + partition_label
    os.makedirs(partition_path)
    os.makedirs(partition_path + '/en.dev')
    os.makedirs(partition_path + '/en.test')
    os.makedirs(partition_path + '/en.train')
    
    with open(partition_path + '/en.dev/parses.json', 'w') as dev_parses_file, \
        open(partition_path + '/en.test/parses.json', 'w') as test_parses_file, \
        open(partition_path + '/en.train/parses.json', 'w') as train_parses_file, \
        open(partition_path + '/en.dev/relations.json', 'w') as dev_relations_file, \
        open(partition_path + '/en.test/relations.json', 'w') as test_relations_file, \
        open(partition_path + '/en.train/relations.json', 'w') as train_relations_file:
        
        # parses
        dev_parses_file.write(json.dumps(total_parses[2]))
        test_parses_file.write(json.dumps(total_parses[1]))
        train_parses_file.write(json.dumps(total_parses[0]))
                                
        for i, total_relations_partition in enumerate(total_relations):
            for relation in total_relations_partition:
                if i == 0:
                    train_relations_file.write(json.dumps(relation))
                    train_relations_file.write('\n')
                if i == 1:
                    test_relations_file.write(json.dumps(relation))
                    test_relations_file.write('\n')
                if i == 2:
                    dev_relations_file.write(json.dumps(relation))
                    dev_relations_file.write('\n')


In [23]:
mix_data(conll_path, twitter_path, output_path, None, 'NoTokenLimit')
mix_data(conll_path, twitter_path, output_path, 20000, '20k')
mix_data(conll_path, twitter_path, output_path, 30000, '30k')

dict_keys(['wsj_0200', 'wsj_0201', 'wsj_0202', 'wsj_0204', 'wsj_0207', 'wsj_0208', 'wsj_0209', 'wsj_0210', 'wsj_0211', 'wsj_0213', 'wsj_0214', 'wsj_0215', 'wsj_0216', 'wsj_0218', 'wsj_0219', 'wsj_0220', 'wsj_0221', 'wsj_0222', 'wsj_0223', 'wsj_0224', 'wsj_0225', 'wsj_0227', 'wsj_0228', 'wsj_0229', 'wsj_0230', 'wsj_0231', 'wsj_0232', 'wsj_0233', 'wsj_0235', 'wsj_0236', 'wsj_0237', 'wsj_0238', 'wsj_0239', 'wsj_0240', 'wsj_0241', 'wsj_0242', 'wsj_0243', 'wsj_0244', 'wsj_0245', 'wsj_0246', 'wsj_0247', 'wsj_0248', 'wsj_0250', 'wsj_0251', 'wsj_0252', 'wsj_0253', 'wsj_0254', 'wsj_0255', 'wsj_0256', 'wsj_0257', 'wsj_0258', 'wsj_0259', 'wsj_0260', 'wsj_0261', 'wsj_0262'])
TOKEN COUNT:  20101
dict_keys(['wsj_0200', 'wsj_0201', 'wsj_0202', 'wsj_0204', 'wsj_0207', 'wsj_0208', 'wsj_0209', 'wsj_0210', 'wsj_0211', 'wsj_0213', 'wsj_0214', 'wsj_0215', 'wsj_0216', 'wsj_0218', 'wsj_0219', 'wsj_0220', 'wsj_0221', 'wsj_0222', 'wsj_0223', 'wsj_0224', 'wsj_0225', 'wsj_0227', 'wsj_0228', 'wsj_0229', 'wsj_0230

In [24]:
mix_data(conll_path, twitter_path, output_path, 465000, '465k')

dict_keys(['wsj_0200', 'wsj_0201', 'wsj_0202', 'wsj_0204', 'wsj_0207', 'wsj_0208', 'wsj_0209', 'wsj_0210', 'wsj_0211', 'wsj_0213', 'wsj_0214', 'wsj_0215', 'wsj_0216', 'wsj_0218', 'wsj_0219', 'wsj_0220', 'wsj_0221', 'wsj_0222', 'wsj_0223', 'wsj_0224', 'wsj_0225', 'wsj_0227', 'wsj_0228', 'wsj_0229', 'wsj_0230', 'wsj_0231', 'wsj_0232', 'wsj_0233', 'wsj_0235', 'wsj_0236', 'wsj_0237', 'wsj_0238', 'wsj_0239', 'wsj_0240', 'wsj_0241', 'wsj_0242', 'wsj_0243', 'wsj_0244', 'wsj_0245', 'wsj_0246', 'wsj_0247', 'wsj_0248', 'wsj_0250', 'wsj_0251', 'wsj_0252', 'wsj_0253', 'wsj_0254', 'wsj_0255', 'wsj_0256', 'wsj_0257', 'wsj_0258', 'wsj_0259', 'wsj_0260', 'wsj_0261', 'wsj_0262', 'wsj_0263', 'wsj_0264', 'wsj_0265', 'wsj_0266', 'wsj_0267', 'wsj_0268', 'wsj_0269', 'wsj_0270', 'wsj_0271', 'wsj_0272', 'wsj_0273', 'wsj_0274', 'wsj_0275', 'wsj_0276', 'wsj_0277', 'wsj_0278', 'wsj_0279', 'wsj_0280', 'wsj_0281', 'wsj_0282', 'wsj_0283', 'wsj_0284', 'wsj_0286', 'wsj_0287', 'wsj_0288', 'wsj_0289', 'wsj_0290', 'wsj_

TOKEN COUNT:  466540
